In [ ]:
%pylab inline

In [ ]:
NCHANNELS = 16

In [ ]:
fname = 'test1_11_arc_18_01.17h36m19__bias_notholding.bin'

In [ ]:
dat = fromfile(fname)
dat.shape

nsamp = int(floor(dat.shape[0]/NCHANNELS))

captured = dat[ :(nsamp*NCHANNELS) ].reshape(nsamp,NCHANNELS)

In [ ]:
plot(captured)